In [1]:
%tensorflow_version 1.x
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

TensorFlow 1.x selected.
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/tfidf-paper/")

In [0]:

import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# extracting features
#df is data
#name is one of Abstract, Title, Keywords
def chi(df,name): 
    
    Dic=[]         #Dictionary contains chi-value of every word belong to certain category 
    caTegory=[]
    for i in df.category:
      if i not in caTegory:
        caTegory.append(i)
    for i in caTegory:
      #print(i)
      doc=np.array(df[name][df.category==i])
      doc1=np.array(df[name][df.category!=i])
      vect =  CountVectorizer(binary=True,lowercase = True,ngram_range=(1,1))             
      X_dtm = vect.fit_transform(doc)
      vect1= CountVectorizer(binary=True,lowercase = True,ngram_range=(1,1),vocabulary=vect.get_feature_names())  
      
      A = np.sum(X_dtm.toarray(),axis=0)
      B=vect1.fit_transform(doc1)
      B = np.sum(B.toarray(),axis=0)
      C=len(train.category[train.category==i])*np.ones([1,len(A)])
      D=len(train.category[train.category!=i])*np.ones([1,len(B)])
      C=C[0]-A
      D=D[0]-B
      result=((A+B+C+D)*(A*D-B*C)**2)/((A+C)*(B+D)*(A+B)*(C+D))
     
      Dic.append(dict(zip(vect.get_feature_names(),result)))
      del doc,doc1,vect1,vect,X_dtm,A,B,C,D,result
    return Dic

#save and load
import pickle
def save_obj(obj, name ):
    with open( name +'.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [0]:
import pandas as pd
path="/opt/dac_research/Son/old_method/"
train=pd.read_csv(path+"Train.csv")
valid=pd.read_csv(path+"Valid.csv")
test=pd.read_csv(path+"Test.csv")

chi_abstract=chi(train,"abstract")
chi_title=chi(train,"title")
chi_key=chi(train,"keywords")

In [0]:
def get_feature(df1,Dic):
    caTegory=[]
    for i in df1.category:
      if i not in caTegory:
        caTegory.append(i)
    
    Word=[]
    for i in range(0,len(caTegory)):
      j=0
    #get 50 top words 
      for dic,val in {k: v for k, v in sorted(Dic[i].items(), key=lambda item: item[1],reverse=True)}.items():
        j=j+1
        Word.append(dic)
        if j >=50:
          break
    Word=__builtins__.list(set(Word)) 
    return Word


In [0]:
abstract=get_feature(train,chi_abstract)
title=get_feature(train,chi_title)
key=get_feature(train,chi_key)

In [0]:
caTegory=[]
for i in train.category:
    if i not in caTegory:
        caTegory.append(i)

In [0]:
def tf_idf(train,test,valid,name,chi_name): #name is a string
#train
    tfidf = TfidfVectorizer(vocabulary=chi_name, ngram_range=(1,1))
    tfidf.fit_transform(__builtins__.list(train[name]))
    Vector_Train=tfidf.transform(__builtins__.list(train[name]))
    Vector_train=Vector_Train.toarray()

#test
    Vector_Test=tfidf.transform(__builtins__.list(test[name]))
    Vector_test=Vector_Test.toarray()


#valid
    Vector_Valid=tfidf.transform(__builtins__.list(valid[name]))
    Vector_valid=Vector_Valid.toarray()
    return Vector_train, Vector_test, Vector_valid

In [0]:
Vector_abstract_train,Vector_abstract_test,Vector_abstract_valid=tf_idf(train,test,valid,"abstract",abstract)

Vector_title_train,Vector_title_test,Vector_title_valid=tf_idf(train,test,valid,"title",title)

Vector_keywords_train,Vector_keywords_test,Vector_keywords_valid=tf_idf(train,test,valid,"keywords",key)